# Lecture 3-6 LAB: Filtering in the Frequency Domain

## 0.- Initialize filesystem and libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pydicom

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pydicom

## 1.- Filtering in the Frequency Domain

Apply different filters (low-, high-, band-pass, Gaussian and Butterworth, with different orders) in the frequency domain to the image `X-ray_5.png'.

### Gaussian Low-pass Filter (assigned task)

### Gaussian High-pass Filter (assigned task)

### Gaussian Band-pass Filter

In [ ]:
# Gaussian low-pass filter
def gaussian_low_pass(rows, cols, cutoff):
    x, y = np.meshgrid(np.linspace(-1, 1, cols), np.linspace(-1, 1, rows))
    d = np.sqrt(x**2 + y**2)  # Distance from the center
    H = np.exp(-(d**2) / (2 * (cutoff**2)))  # Gaussian low-pass filter formula
    return H

# Gaussian high-pass filter
def gaussian_high_pass(rows, cols, cutoff):
    low_pass = gaussian_low_pass(rows, cols, cutoff)
    return 1 - low_pass  # Gaussian high-pass filter is 1 - low-pass filter

# Gaussian band-pass filter
def gaussian_band_pass(rows, cols, low_cutoff, high_cutoff):
    low_pass = gaussian_low_pass(rows, cols, high_cutoff)
    high_pass = gaussian_high_pass(rows, cols, low_cutoff)
    band_pass = low_pass * high_pass  # Multiply the low-pass and high-pass filters
    return band_pass

# Load the image
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_5.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = image.astype(np.float32)  # Convert to float32 format before applying FFT

# Perform the 2D FFT on the image
dft = cv2.dft(np.float32(image), flags=cv2.DFT_COMPLEX_OUTPUT)
dft_shifted = np.fft.fftshift(dft)

# Create the Gaussian band-pass filter
low_cutoff = 0.05  # Low cutoff frequency (in normalized units)
high_cutoff = 0.1  # High cutoff frequency (in normalized units)
rows, cols = image.shape  # Rows and columns of the image
gaussian_band_pass_filter = gaussian_band_pass(rows, cols, low_cutoff, high_cutoff)

# Apply the Gaussian band-pass filter in the frequency domain
filtered_real_band_pass = dft_shifted[:, :, 0] * gaussian_band_pass_filter
filtered_imaginary_band_pass = dft_shifted[:, :, 1] * gaussian_band_pass_filter
filtered_dft_band_pass = np.dstack((filtered_real_band_pass, filtered_imaginary_band_pass))

# Shift back the filtered FFT and compute the inverse FFT
filtered_dft_band_pass_shifted_back = np.fft.ifftshift(filtered_dft_band_pass)
filtered_image_band_pass = cv2.idft(filtered_dft_band_pass_shifted_back)
filtered_image_magnitude_band_pass = cv2.magnitude(filtered_image_band_pass[:, :, 0], filtered_image_band_pass[:, :, 1])

# Compute the magnitude of the 2D FFT for the original and filtered images
magnitude_spectrum_original = cv2.magnitude(dft_shifted[:, :, 0], dft_shifted[:, :, 1])
filtered_dft_band_pass_magnitude = cv2.magnitude(filtered_dft_band_pass[:, :, 0], filtered_dft_band_pass[:, :, 1])

# Plot the results
plt.figure(figsize=(10, 8))

# Original image
plt.subplot(2, 3, 1)
plt.imshow(np.uint8(image), cmap='gray')
plt.title("Original image")
plt.axis('off')

# 2D FFT of the original image (log scale)
plt.subplot(2, 3, 2)
plt.imshow(np.log1p(magnitude_spectrum_original), cmap='gray')
plt.title("2D FFT of original image\n (log scale)")
plt.axis('off')

# Gaussian Band-pass filter
plt.subplot(2, 3, 3)
plt.imshow(gaussian_band_pass_filter, cmap='gray')
plt.title("Gaussian band-pass filter")
plt.axis('off')

# Band-pass filtered image (normalized)
plt.subplot(2, 3, 4)
plt.imshow(cv2.normalize(filtered_image_magnitude_band_pass, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cmap='gray')
plt.title("Band-pass filtered image\n (normalized)")
plt.axis('off')

# 2D FFT of the band-pass filtered image (log scale)
plt.subplot(2, 3, 5)
plt.imshow(np.log1p(filtered_dft_band_pass_magnitude), cmap='gray')
plt.title("2D FFT of band-pass filtered\n image (log scale)")
plt.axis('off')

plt.tight_layout()
plt.show()

### Butterworth Low- and High-pass Filters

In [ ]:
# Butterworth low-pass filter
def butterworth_low_pass(rows, cols, cutoff, order):
    x, y = np.meshgrid(np.linspace(-1, 1, cols), np.linspace(-1, 1, rows))  # Create a meshgrid
    d = np.sqrt(x**2 + y**2)  # Distance from the center
    H = 1 / (1 + (d / cutoff)**(2 * order))  # Low-pass filter formula
    return H

# Butterworth high-pass filter
def butterworth_high_pass(rows, cols, cutoff, order):
    x, y = np.meshgrid(np.linspace(-1, 1, cols), np.linspace(-1, 1, rows))  # Create a meshgrid
    d = np.sqrt(x**2 + y**2)  # Distance from the center
    H = 1 / (1 + (cutoff / d)**(2 * order))  # High-pass filter formula
    return H

# Step 1: Load the image
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_5.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = image.astype(np.float32)  # Convert to float32 format before applying FFT

# Step 2: Perform the 2D FFT on the image
dft = cv2.dft(np.float32(image), flags=cv2.DFT_COMPLEX_OUTPUT)
dft_shifted = np.fft.fftshift(dft)

# Step 3: Create the Butterworth filters
cutoff = 0.1  # Cutoff frequency (in normalized units)
order = 1  # Order of the Butterworth filter (higher values = sharper cutoff)
rows, cols = image.shape
butterworth_low_pass_filter = butterworth_low_pass(rows, cols, cutoff, order)
butterworth_high_pass_filter = butterworth_high_pass(rows, cols, cutoff, order)

# Step 4: Apply the Butterworth filters in the frequency domain
# Low-pass filter
filtered_real_low_pass = dft_shifted[:, :, 0] * butterworth_low_pass_filter
filtered_imaginary_low_pass = dft_shifted[:, :, 1] * butterworth_low_pass_filter
filtered_dft_low_pass = np.dstack((filtered_real_low_pass, filtered_imaginary_low_pass))
# High-pass filter
filtered_real_high_pass = dft_shifted[:, :, 0] * butterworth_high_pass_filter
filtered_imaginary_high_pass = dft_shifted[:, :, 1] * butterworth_high_pass_filter
filtered_dft_high_pass = np.dstack((filtered_real_high_pass, filtered_imaginary_high_pass))

# Step 5: Shift back both filtered FFTs and compute the inverse FFTs
# Low-pass filter
filtered_dft_low_pass_shifted_back = np.fft.ifftshift(filtered_dft_low_pass)
filtered_image_low_pass = cv2.idft(filtered_dft_low_pass_shifted_back)
# High-pass filter
filtered_dft_high_pass_shifted_back = np.fft.ifftshift(filtered_dft_high_pass)
filtered_image_high_pass = cv2.idft(filtered_dft_high_pass_shifted_back)

# Step 6: Compute the magnitude of the original and filtered images
magnitude_spectrum_original = cv2.magnitude(dft_shifted[:, :, 0], dft_shifted[:, :, 1])
filtered_image_magnitude_low_pass = cv2.magnitude(filtered_image_low_pass[:, :, 0], filtered_image_low_pass[:, :, 1])
filtered_image_magnitude_high_pass = cv2.magnitude(filtered_image_high_pass[:, :, 0], filtered_image_high_pass[:, :, 1])

# Step 7: Plot the results
plt.figure(figsize=(8, 10))

# Original image
plt.subplot(3, 2, 1)
plt.imshow(image, cmap='gray')
plt.title("Original image")
plt.axis('off')

# 2D FFT of the original image
plt.subplot(3, 2, 2)
plt.imshow(np.log1p(magnitude_spectrum_original), cmap='gray')
plt.title("2D FFT of original image (log scale)")
plt.axis('off')

# Butterworth Low-pass filter
plt.subplot(3, 2, 3)
plt.imshow(butterworth_low_pass_filter, cmap='gray')
plt.title("Butterworth low-pass filter")
plt.axis('off')

# Low-pass filtered image (normalized)
plt.subplot(3, 2, 4)
plt.imshow(np.uint8(cv2.normalize(filtered_image_magnitude_low_pass, None, 0, 255, cv2.NORM_MINMAX)), cmap='gray')
plt.title("Low-pass filtered image (normalized)")
plt.axis('off')

# Butterworth High-pass filter
plt.subplot(3, 2, 5)
plt.imshow(butterworth_high_pass_filter, cmap='gray')
plt.title("Butterworth high-pass filter")
plt.axis('off')

# High-pass filtered image (normalized)
plt.subplot(3, 2, 6)
plt.imshow(np.uint8(cv2.normalize(filtered_image_magnitude_high_pass, None, 0, 255, cv2.NORM_MINMAX)), cmap='gray')
plt.title("High-pass filtered image (normalized)")
plt.axis('off')

plt.tight_layout()
plt.show()

### Butterworth Band-pass Filter

In [ ]:
# Butterworth low-pass filter
def butterworth_low_pass(rows, cols, cutoff, order):
    x, y = np.meshgrid(np.linspace(-1, 1, cols), np.linspace(-1, 1, rows))
    d = np.sqrt(x**2 + y**2)  # Distance from the center
    H = 1 / (1 + (d / cutoff)**(2 * order))  # Low-pass filter formula
    return H

# Butterworth high-pass filter
def butterworth_high_pass(rows, cols, cutoff, order):
    x, y = np.meshgrid(np.linspace(-1, 1, cols), np.linspace(-1, 1, rows))
    d = np.sqrt(x**2 + y**2)  # Distance from the center
    H = 1 / (1 + (cutoff / d)**(2 * order))  # High-pass filter formula
    return H

# Butterworth band-pass filter
def butterworth_band_pass(rows, cols, low_cutoff, high_cutoff, order):
    low_pass = butterworth_low_pass(rows, cols, high_cutoff, order)
    high_pass = butterworth_high_pass(rows, cols, low_cutoff, order)
    band_pass = low_pass * high_pass
    return band_pass

# Load the image
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_5.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = image.astype(np.float32)  # Convert to float32 format before applying FFT

# Perform the 2D FFT on the image
dft = cv2.dft(np.float32(image), flags=cv2.DFT_COMPLEX_OUTPUT)
dft_shifted = np.fft.fftshift(dft)

# Create the Butterworth band-pass filter
low_cutoff = 0.05  # Low cutoff frequency (in normalized units)
high_cutoff = 0.1  # High cutoff frequency (in normalized units)
order = 1  # Order of the Butterworth filter
rows, cols = image.shape  # Rows and columns of the image
butterworth_band_pass_filter = butterworth_band_pass(rows, cols, low_cutoff, high_cutoff, order)

# Apply the Butterworth band-pass filter in the frequency domain
filtered_real_band_pass = dft_shifted[:, :, 0] * butterworth_band_pass_filter
filtered_imaginary_band_pass = dft_shifted[:, :, 1] * butterworth_band_pass_filter
filtered_dft_band_pass = np.dstack((filtered_real_band_pass, filtered_imaginary_band_pass))

# Shift back the filtered FFT and compute the inverse FFT
filtered_dft_band_pass_shifted_back = np.fft.ifftshift(filtered_dft_band_pass)
filtered_image_band_pass = cv2.idft(filtered_dft_band_pass_shifted_back)
filtered_image_magnitude_band_pass = cv2.magnitude(filtered_image_band_pass[:, :, 0], filtered_image_band_pass[:, :, 1])

# Compute the magnitude of the 2D FFT for the original and filtered images
magnitude_spectrum_original = cv2.magnitude(dft_shifted[:, :, 0], dft_shifted[:, :, 1])
filtered_dft_band_pass_magnitude = cv2.magnitude(filtered_dft_band_pass[:, :, 0], filtered_dft_band_pass[:, :, 1])

# Plot the results
plt.figure(figsize=(10, 8))

# Original image
plt.subplot(2, 3, 1)
plt.imshow(np.uint8(image), cmap='gray')
plt.title("Original image")
plt.axis('off')

# 2D FFT of the original image (log scale)
plt.subplot(2, 3, 2)
plt.imshow(np.log1p(magnitude_spectrum_original), cmap='gray')
plt.title("2D FFT of original image\n (log scale)")
plt.axis('off')

# Butterworth Band-pass filter
plt.subplot(2, 3, 3)
plt.imshow(butterworth_band_pass_filter, cmap='gray')
plt.title("Butterworth band-pass filter")
plt.axis('off')

# Band-pass filtered image (normalized)
plt.subplot(2, 3, 4)
plt.imshow(cv2.normalize(filtered_image_magnitude_band_pass, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cmap='gray')
plt.title("Band-pass filtered image\n (normalized)")
plt.axis('off')

# 2D FFT of the band-pass filtered image (log scale)
plt.subplot(2, 3, 5)
plt.imshow(np.log1p(filtered_dft_band_pass_magnitude), cmap='gray')
plt.title("2D FFT of band-pass filtered\n image (log scale)")
plt.axis('off')

plt.tight_layout()
plt.show()